In [1]:
import pandas as pd
import os

In [2]:
#random notes and settings
#set exporter.fhir.export = false
#set exporter.csv.export = true

#encounters want reason code == 55680006 (drug overdose)
#encounters want description == 'Death Certification'

#run_synthea -p 10000 -s 10000 -cs 12345 -m opioid_addiction Maine Bangor

#^^this command generates 10000 people (-p) with the seed 10000 (-s) and the provider seed of 12345 (-cs) using the opiod_addition module (-m) in Bangor, ME

In [3]:
def makeEncountersDF(path, seeds):
    '''
    Go to the path and grab all encounters.csv and put them in one file with an extra column for seed
    '''
    df = pd.DataFrame()
    for seed in seeds:
        try:
            encounters = pd.read_csv(os.path.join(path, 'bangor_s' + str(seed), 'encounters.csv'), dtype=str)
            encounters['seed'] = seed
            df = df.append(encounters)
        except:
            print('File for seed', str(seed), 'does not exist...skipping')
    return df

In [4]:
def getODEncounters(df):
    '''
    Return all drug overdose encounters (reason code 5568006) from a given encounters dataframe
    This will include overdose deaths as well -- description -- 'Death Certification'
    '''
    data = df[df['REASONCODE'] == '55680006']
    return data

In [5]:
def getODDeaths(df):
    '''
    Return all drug overdose deaths (reason code 5568006) from a given encounters dataframe
    DESCRIPTION == 'Death Certification' and REASONCODE == '5568006'
    '''
    data = df[(df['DESCRIPTION'] == 'Death Certification') & (df['REASONCODE'] == '55680006')]
    return data

In [6]:
def getODstats(df):
    '''
    get patient level sample statistics on probability of death per drug overdose ED visit
    '''
    #getting all overdose encouunters
    od_enc = getODEncounters(df)
    od_enc = od_enc.groupby(['PATIENT','seed'], as_index=False)['REASONCODE'].count().rename(columns={'REASONCODE':'OD_count'})
    #getting all overdose deaths
    od_death = getODDeaths(df)
    od_death = od_death.groupby(['PATIENT','seed'], as_index=False)['REASONCODE'].count().rename(columns={'REASONCODE':'OD_death'})
    #joining the above two dataframes
    od = pd.merge(od_enc, od_death, how='left', on=['PATIENT', 'seed']).fillna(0)
    #calculating patient level probability of death from overdose encounter
    od['prDeath'] = od['OD_death']/od['OD_count']
    #making column for weight of patient to calculate weighted average probability
    od['weight'] = od['OD_count']/sum(od['OD_count'])
    #weighted pr(death) -- can sum this column to get weighted sample pr(death)
    od['weightedPrDeath'] = od['weight']*od['prDeath']
    
    return od

In [7]:
path = r'C:\repos\Synthea\output'
#seeds = [10000, 13370, 22222, 23123, 33555, 39093, 45000, 51327, 65888, 74982]
#seeds = [12345]
seeds = [22222]

#pull in data
df = makeEncountersDF(path, seeds)

In [8]:
#calculate overdose stats
od_df = getODstats(df)

print(od_df['prDeath'].mean())

print(od_df['weightedPrDeath'].sum())

0.020320082728193224
0.020590814469521542


In [13]:
len(df['PATIENT'].unique())

4560698

In [9]:
od_df

,PATIENT,seed,OD_count,OD_death,prDeath,weight,weightedPrDeath
0,00000c94-212f-7940-b8c1-692423cfad98,22222,4,1.0,0.250,0.000005,0.000001
1,00006d99-ad13-aae4-c40a-002954805bdf,22222,2,1.0,0.500,0.000002,0.000001
2,00007137-723e-ff4b-3ab4-01f180988500,22222,4,0.0,0.000,0.000005,0.000000
3,00009bfd-2bca-3101-730c-a218315c191e,22222,5,0.0,0.000,0.000006,0.000000
4,0000a8f2-d6a4-ab21-294f-c13ac2f45034,22222,8,1.0,0.125,0.000009,0.000001
...,...,...,...,...,...,...,...
229212,fffef610-5bd7-c85a-f0e7-051b8a7a522e,22222,5,0.0,0.000,0.000006,0.000000
229213,ffff0ae8-7753-176d-bf7a-f5ee0a0cda7c,22222,4,0.0,0.000,0.000005,0.000000
229214,ffff1883-c1c0-4a88-2ed9-d71aaf5449c0,22222,3,0.0,0.000,0.000003,0.000000
229215,ffff5e59-6e15-a258-444a-ea4a47d92a64,22222,4,0.0,0.000,0.000005,0.000000


## The number of prescriptions per person, per year, by drug code & description

In [8]:
def makeMedicationsDF(path, seeds):
    '''
    Go to the path and grab all encounters.csv and put them in one file with an extra column for seed
    '''
    df = pd.DataFrame()
    for seed in seeds:
        try:
            medications = pd.read_csv(os.path.join(path, 'bangor_s' + str(seed), 'medications.csv'), dtype=str)
            medications['seed'] = seed
            df = df.append(medications)
        except:
            print('File for seed', str(seed), 'does not exist...skipping')
    return df

In [9]:
#path = r'\\lmi.org\Data\Ser_Del\HlthMgmt\Civ\RstricOpen\SyntheaChallenge\data'
path = r'C:\repos\Synthea\output'
seeds = [10000, 13370, 22222, 23123, 33555, 39093, 45000, 51327, 65888, 74982]

#pull in data
df = makeMedicationsDF(path, seeds)

In [10]:
df

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION,seed
0,2010-07-21T21:17:12Z,2011-07-21T21:17:12Z,cb8ec237-0759-b2e4-e89a-79df87ae36a0,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,e554e4c1-e02a-6842-6d84-f33c68e67330,807283,Mirena 52 MG Intrauterine System,681.13,621.13,12,8173.56,NaN,NaN,10000
1,2011-09-26T06:40:20Z,2012-09-20T06:40:20Z,956e9fa8-ed18-234d-f77b-bdc9c93fe5da,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,eb3d2956-efa3-e324-73af-d389237b1388,749762,Seasonique 91 Day Pack,25.97,0.00,12,311.64,NaN,NaN,10000
2,2001-06-26T12:17:19Z,2002-07-02T12:17:19Z,a36a5420-69d5-6356-2a07-d0fa177c59ef,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,9af04fe6-e9ce-4085-20ce-c50b99a1c53c,308136,amLODIPine 2.5 MG Oral Tablet,0.01,0.00,371,3.71,59621000,Hypertension,10000
3,2001-07-26T12:17:19Z,2002-07-02T12:17:19Z,a36a5420-69d5-6356-2a07-d0fa177c59ef,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,87d37361-f626-29b4-820d-46972c1721ef,314076,lisinopril 10 MG Oral Tablet,0.01,0.00,341,3.41,59621000,Hypertension,10000
4,2002-07-02T12:17:19Z,2003-07-08T12:17:19Z,a36a5420-69d5-6356-2a07-d0fa177c59ef,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,d2ab512a-d049-23b6-078e-e8b57728abd1,314076,lisinopril 10 MG Oral Tablet,0.01,0.00,371,3.71,59621000,Hypertension,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955697,2020-05-30T15:54:09Z,2021-06-11T15:50:14Z,414e1d64-2b15-4026-82f9-0b9db7193669,b3221cfc-24fb-339e-823d-bc4136cbc4ed,53cff874-3a38-fd4d-bab9-5d20ed5a8282,197361,Amlodipine 5 MG Oral Tablet,36.18,0.00,12,434.16,NaN,NaN,74982
1955698,2021-06-11T15:50:14Z,NaN,414e1d64-2b15-4026-82f9-0b9db7193669,b3221cfc-24fb-339e-823d-bc4136cbc4ed,a0e28916-e357-afc9-a9c1-79d3a5ae1989,309362,Clopidogrel 75 MG Oral Tablet,38.95,0.00,1,38.95,NaN,NaN,74982
1955699,2021-06-11T15:50:14Z,NaN,414e1d64-2b15-4026-82f9-0b9db7193669,b3221cfc-24fb-339e-823d-bc4136cbc4ed,a0e28916-e357-afc9-a9c1-79d3a5ae1989,705129,Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray,185.59,0.00,1,185.59,NaN,NaN,74982
1955700,2021-06-11T15:50:14Z,NaN,414e1d64-2b15-4026-82f9-0b9db7193669,b3221cfc-24fb-339e-823d-bc4136cbc4ed,a0e28916-e357-afc9-a9c1-79d3a5ae1989,312961,Simvastatin 20 MG Oral Tablet,33.43,0.00,1,33.43,NaN,NaN,74982


In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 19118720 entries, 0 to 1955701
Data columns (total 14 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   START              object
 1   STOP               object
 2   PATIENT            object
 3   PAYER              object
 4   ENCOUNTER          object
 5   CODE               object
 6   DESCRIPTION        object
 7   BASE_COST          object
 8   PAYER_COVERAGE     object
 9   DISPENSES          object
 10  TOTALCOST          object
 11  REASONCODE         object
 12  REASONDESCRIPTION  object
 13  seed               int64 
dtypes: int64(1), object(13)
memory usage: 2.1+ GB


In [12]:
df['YEAR'] = df['START'].str.slice(stop=4)

In [13]:
df_grouped = df.groupby(['seed', 'PATIENT', 'YEAR', 'CODE', 'DESCRIPTION'])['ENCOUNTER'].count() \
    .reset_index(name='Number of Prescriptions')

In [14]:
df_grouped.head()

,seed,PATIENT,YEAR,CODE,DESCRIPTION,Number of Prescriptions
0,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1982,314076,lisinopril 10 MG Oral Tablet,1
1,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1983,314076,lisinopril 10 MG Oral Tablet,2
2,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1984,314076,lisinopril 10 MG Oral Tablet,1
3,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1985,314076,lisinopril 10 MG Oral Tablet,1
4,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1986,314076,lisinopril 10 MG Oral Tablet,1


In [15]:
df_grouped.to_csv("prescription_info_including_seeds.csv", index=False)

In [19]:
df_grouped.head()

,seed,PATIENT,YEAR,CODE,DESCRIPTION,Number of Prescriptions
0,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1982,314076,lisinopril 10 MG Oral Tablet,1
1,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1983,314076,lisinopril 10 MG Oral Tablet,2
2,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1984,314076,lisinopril 10 MG Oral Tablet,1
3,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1985,314076,lisinopril 10 MG Oral Tablet,1
4,10000,0002f32f-58e0-7566-8e67-61f990b66a0a,1986,314076,lisinopril 10 MG Oral Tablet,1


In [27]:
listOfDrugs = df_grouped.groupby(['CODE','DESCRIPTION'])['Number of Prescriptions'].sum().reset_index()

In [30]:
listOfDrugs.to_csv("prescription_list.csv", index=False)

## How many opioids were prescribed by year divided by total number of people in the simulation in the year

### need to modify this to keep the seed and use the list of opioids that Maureen is filling out

In [16]:
df_total_people_by_year = df.groupby(['YEAR'])['PATIENT'].nunique().reset_index(name='Total Unique Patients')
df_opioids = df[df['DESCRIPTION'].str.lower().str.contains('oxycodone')]
df_num_opioids_by_year = df_opioids.groupby(['YEAR'])['DESCRIPTION'].count().reset_index(name='Number of Opioids')

In [17]:
df_opioids_per_capita = df_num_opioids_by_year.merge(df_total_people_by_year, how='left', on='YEAR')
df_opioids_per_capita['Per Capita'] = df_opioids_per_capita['Number of Opioids'] / df_opioids_per_capita['Total Unique Patients']
df_opioids_per_capita

,YEAR,Number of Opioids,Total Unique Patients,Per Capita
0,1922,2,293,0.006826
1,1923,12,322,0.037267
2,1924,23,387,0.059432
3,1925,31,401,0.077307
4,1926,41,437,0.093822
...,...,...,...,...
95,2017,30872,204151,0.151221
96,2018,30820,206004,0.149609
97,2019,30448,207622,0.146651
98,2020,29287,223423,0.131083


In [18]:
df_opioids_per_capita.to_csv("opioids_prescription_per_capita.csv", index=False)